Classfication algorithm based on Bayes' theorem

Describes the probablity of an event occuring based on conditions

Conditions must be related to the event

Built by assigning labels to instances, which are represented as vectors of features

Assumes that value of any feature is independent to value of any other features (independence assumption)

Algorithm takes into consideration the features independently before giving the probability as an outcome, do not consider 
correlations that may occur between the features

P(A happening if B happens)=P(B happens if A happens).P(A happens independently) / P(B happens independently)

P(A|B)=[P(B|A).P(A)/P(B)]

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
df=pd.read_csv('play.csv')
df.head()

,Outlook,Temperature,Humidity,Windy,Play
0,sunny,hot,high,F,no
1,sunny,hot,high,T,no
2,overcast,hot,high,F,yes
3,rainy,mild,high,F,yes
4,rainy,cool,normal,F,yes


In [20]:
#Separating data from the dataset into features and outcome
X=df.drop([df.columns[-1]],axis=1)
Y=df[df.columns[-1]]
print(X.head())
print(Y.head())

    Outlook Temperature Humidity Windy
0     sunny         hot     high     F
1     sunny         hot     high     T
2  overcast         hot     high     F
3     rainy        mild     high     F
4     rainy        cool   normal     F
0     no
1     no
2    yes
3    yes
4    yes
Name: Play, dtype: object


In [21]:
feature=list(X.columns)
feature

['Outlook', 'Temperature', 'Humidity', 'Windy']

In [22]:
#Data to train the model,and number of features for the model
X_train=X
Y_train=Y
train_size=X.shape[0]
num_features=X.shape[1]

In [23]:
print(train_size)
print(num_features)

14
4


In [24]:
#Probability of event occuring
likelihood={}
#Starting point for the prediction
class_prior={}
#Probability of outcome based on avaialble data
pred_prior={}

In [25]:
for i in feature:
    likelihood[i]={}
    pred_prior[i]={}

    #Get unique feature for each feature column
    for fval in np.unique(X_train[i]):
        #Initialize value to 0
        pred_prior[i].update({fval:0})

        #Unique results from training column
        for k in np.unique(Y_train):
            likelihood[i].update({fval+'_'+k:0})
            class_prior.update({k:0})

In [26]:
print(likelihood)
print('')
print(pred_prior)
print('')
print(class_prior)

{'Outlook': {'overcast_no': 0, 'overcast_yes': 0, 'rainy_no': 0, 'rainy_yes': 0, 'sunny_no': 0, 'sunny_yes': 0}, 'Temperature': {'cool_no': 0, 'cool_yes': 0, 'hot_no': 0, 'hot_yes': 0, 'mild_no': 0, 'mild_yes': 0}, 'Humidity': {'high_no': 0, 'high_yes': 0, 'normal_no': 0, 'normal_yes': 0}, 'Windy': {'F_no': 0, 'F_yes': 0, 'T_no': 0, 'T_yes': 0}}

{'Outlook': {'overcast': 0, 'rainy': 0, 'sunny': 0}, 'Temperature': {'cool': 0, 'hot': 0, 'mild': 0}, 'Humidity': {'high': 0, 'normal': 0}, 'Windy': {'F': 0, 'T': 0}}

{'no': 0, 'yes': 0}


In [27]:
for i in np.unique(Y_train):
    count=sum(Y_train==i)
    class_prior[i]=count/train_size

In [28]:
class_prior

{'no': 0.35714285714285715, 'yes': 0.6428571428571429}

In [29]:
for i in feature:
    for j in np.unique(Y_train):
        ocount=sum(Y_train==j)
        likelihood1=X_train[i][Y_train[Y_train==j].index.values.tolist()].value_counts().to_dict()
        print(likelihood1)
        for fval,cnt in likelihood1.items():
            likelihood[i][fval+'_'+j]=cnt/ocount

{'sunny': 3, 'rainy': 2}
{'overcast': 4, 'rainy': 3, 'sunny': 2}
{'hot': 2, 'mild': 2, 'cool': 1}
{'mild': 4, 'cool': 3, 'hot': 2}
{'high': 4, 'normal': 1}
{'normal': 6, 'high': 3}
{'T': 3, 'F': 2}
{'F': 6, 'T': 3}


In [30]:
likelihood

{'Outlook': {'overcast_no': 0,
  'overcast_yes': 0.4444444444444444,
  'rainy_no': 0.4,
  'rainy_yes': 0.3333333333333333,
  'sunny_no': 0.6,
  'sunny_yes': 0.2222222222222222},
 'Temperature': {'cool_no': 0.2,
  'cool_yes': 0.3333333333333333,
  'hot_no': 0.4,
  'hot_yes': 0.2222222222222222,
  'mild_no': 0.4,
  'mild_yes': 0.4444444444444444},
 'Humidity': {'high_no': 0.8,
  'high_yes': 0.3333333333333333,
  'normal_no': 0.2,
  'normal_yes': 0.6666666666666666},
 'Windy': {'F_no': 0.4,
  'F_yes': 0.6666666666666666,
  'T_no': 0.6,
  'T_yes': 0.3333333333333333}}

In [31]:
for i in feature:
    fvals=X_train[i].value_counts().to_dict()
    print(fvals)
    for fval,cnt in fvals.items():
        pred_prior[i][fval]=cnt/train_size

{'sunny': 5, 'rainy': 5, 'overcast': 4}
{'mild': 6, 'hot': 4, 'cool': 4}
{'high': 7, 'normal': 7}
{'F': 8, 'T': 6}


In [32]:
pred_prior

{'Outlook': {'overcast': 0.2857142857142857,
  'rainy': 0.35714285714285715,
  'sunny': 0.35714285714285715},
 'Temperature': {'cool': 0.2857142857142857,
  'hot': 0.2857142857142857,
  'mild': 0.42857142857142855},
 'Humidity': {'high': 0.5, 'normal': 0.5},
 'Windy': {'F': 0.5714285714285714, 'T': 0.42857142857142855}}

In [33]:
final=[]
col=np.array([['rainy','mild','normal','T']])
col=np.array(col)
m=0
p=0
for i in col:
    result={}
    for j in np.unique(Y_train):
        prior=class_prior[j]
        lkl=1
        evidence=1

        for f,fval in zip(feature,i):
            lkl*=likelihood[f][fval+'_'+j]
            evidence*=pred_prior[f][fval]
        post=(lkl*prior+m*p)/(evidence+m)
        result[j]=post
    print(result)
    rsl=max(result,key=lambda x:result[x])
    final.append(rsl)
print(final)

{'no': 0.20906666666666676, 'yes': 0.6452674897119342}
['yes']
